#import

In [ ]:
import copy
import itertools
import numpy as np
import tensorflow as tf
from tensorflow.nn import softmax
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNNCell,Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import RNN, SimpleRNNCell, LSTMCell, GRUCell

In [ ]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import numpy as np
from sklearn.cluster import KMeans

In [ ]:
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, SimpleRNNCell, Conv2D

In [ ]:
import random

#DRNN

In [ ]:
def dRNN(cell, inputs, rate, scope='default'):

    n_steps = len(inputs)
    if rate < 0 or rate >= n_steps:
        raise ValueError('The \'rate\' variable needs to be adjusted.')
    print("Building layer: %s, input length: %d, dilation rate: %d, input dim: %d." % (scope, n_steps, rate, inputs[0].shape[1]))

    # make the length of inputs divide 'rate', by using zero-padding

    EVEN = (n_steps % rate) == 0
    if not EVEN:
        # Create a tensor in shape (batch_size, input_dims), which all elements are zero.
        # This is used for zero padding
        zero_tensor = tf.zeros_like(inputs[0])
        dialated_n_steps = n_steps // rate + 1
        print("=====> %d time points need to be padded. " % (dialated_n_steps * rate - n_steps))
        print("=====> Input length for sub-RNN: %d" % (dialated_n_steps))
        for i_pad in range(dialated_n_steps * rate - n_steps):
            inputs.append(zero_tensor)
    else:
        dialated_n_steps = n_steps // rate
        print("=====> Input length for sub-RNN: %d" % (dialated_n_steps))


    dilated_inputs = [tf.concat(inputs[i * rate:(i + 1) * rate], axis=0) for i in range(dialated_n_steps)]


    # building a dilated RNN with reformatted (dilated) inputs
    dilated_outputs = tf.keras.layers.RNN(cell, return_sequences=True, return_state=False, go_backwards=False,
                                          stateful=False, time_major=True, unroll=False, input_shape=(dialated_n_steps, inputs[0].shape[1]),
                                          name=scope)(tf.stack(dilated_inputs))


    splitted_outputs = tf.split(dilated_outputs, num_or_size_splits=rate, axis=1)
    # unrolled_outputs = [output for sublist in splitted_outputs for output in sublist]
    unrolled_outputs = tf.unstack(tf.concat(splitted_outputs, axis=0))
    # remove padded zeros
    outputs = unrolled_outputs[:n_steps]

    return outputs

#수정Mymodel

In [ ]:
class MyModel(Model):
    def __init__(self,  n_classes):
        super(MyModel, self).__init__()




        self.softmax = tf.keras.activations.softmax
        self.cells = [LSTMCell(20) for _ in range(9)]

        self.classifier = Dense(n_classes, activation='softmax')
        self.dense1 = Dense(128, activation='relu')
        self.flatten=Flatten()
        self.conv2= Conv2D(filters=10, kernel_size=(1, 4), strides=(1, 1), padding='valid')

    def call(self, x):
        for i in range(4,9):
            x = dRNN(self.cells[i], list(x), 2**i)





        x = tf.stack(x, axis=-1)
        x = tf.expand_dims(x, axis=1)

        x =self.conv2(x)
        x=self.flatten(x)

        return  self.classifier(x)

#검증실험1

In [ ]:
#min_dilation==4로 설정
#마지막에 min_dilation에 대한 conv_layer 추가
#drnn 반영

###유사도 기준 데이터 클러스터링 실험1
#유사도가 높은 데이터가 모두 unit 1,2,3,4에 들어감

In [ ]:
#데이터 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28 * 28)) / 255.0
test_images = test_images.reshape((test_images.shape[0], 28 * 28)) / 255.0

#k-means clustering 시행
kmeans = KMeans(n_clusters=2,random_state=2023)
kmeans.fit(train_images)
cluster_labels = kmeans.labels_

clusters = {}
for i, label in enumerate(cluster_labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(i)

restructured_train_images = []
restructured_train_labels = []
for cluster in clusters.values():
    for i in range(0, len(cluster), 4):
        unit = cluster[i:i+4]
        restructured_train_images.extend(train_images[unit])
        restructured_train_labels.extend(train_labels[unit])

restructured_train_images = np.array(restructured_train_images)
restructured_train_labels = np.array(restructured_train_labels)

#텐서로 바꾸기
restructured_train_images = tf.convert_to_tensor(restructured_train_images, dtype=tf.float32)
restructured_train_labels = tf.convert_to_tensor(restructured_train_labels, dtype=tf.uint8)
train_labels_categorical = to_categorical(restructured_train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
restructured_train_images = tf.reshape(restructured_train_images, (60000, 784, 1))
test_images = tf.reshape(test_images, (-1, 784, 1))

In [ ]:
def _rnn_reformat(x, input_dims, n_steps):


    x_ = tf.transpose(x, [1, 0, 2])

    x_ = tf.reshape(x_, [-1, input_dims])

    x_reformat = tf.split(x_, n_steps, 0)

    return x_reformat

x_reformat=_rnn_reformat(restructured_train_images, 1, 28*28)
x_reformat_test=_rnn_reformat(test_images , 1, 28*28)

In [ ]:
model = MyModel( 10)
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_reformat,train_labels_categorical, batch_size=128, epochs=10, validation_data=(x_reformat_test, test_labels_categorical))
model.summary()
print(model.evaluate(x_reformat_test, test_labels_categorical,verbose=1))

313/313 [==============================] - 15s 47ms/step - loss: 0.1248 - accuracy: 0.9690
[0.12478719651699066, 0.968999981880188]


#검증실험2

In [ ]:
###유사도 기준 검증실험2
#unit 1기준 유사도가 대비되는 데이터가 unit 2,3,4에 들어감

In [ ]:
# 데이터 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28 * 28)) / 255.0
test_images = test_images.reshape((test_images.shape[0], 28 * 28)) / 255.0

# k-means 클러스터링
kmeans = KMeans(n_clusters=5, random_state=2023)
kmeans.fit(train_images)
cluster_labels = kmeans.labels_

clusters = {}
for i, label in enumerate(cluster_labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(i)

restructured_train_images = []
restructured_train_labels = []

# 각 클러스터에서 동일한 개수의 데이터를 선택하여 재구성
max_length = max(len(clusters[i]) for i in range(5))

for i in range(max_length):
    for cluster_name in range(5):  # 0, 1, 2, 3, 4
        cluster = clusters[cluster_name]
        if i < len(cluster):
            unit = cluster[i]
            restructured_train_images.append(train_images[unit])
            restructured_train_labels.append(train_labels[unit])

restructured_train_images = np.array(restructured_train_images)
restructured_train_labels = np.array(restructured_train_labels)

# 텐서로 변환
restructured_train_images = tf.convert_to_tensor(restructured_train_images, dtype=tf.float32)
restructured_train_labels = tf.convert_to_tensor(restructured_train_labels, dtype=tf.uint8)
train_labels_categorical = to_categorical(restructured_train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
restructured_train_images = tf.reshape(restructured_train_images, (60000, 784, 1))
test_images = tf.reshape(test_images, (-1, 784, 1))

def _rnn_reformat(x, input_dims, n_steps):


    x_ = tf.transpose(x, [1, 0, 2])

    x_ = tf.reshape(x_, [-1, input_dims])

    x_reformat = tf.split(x_, n_steps, 0)

    return x_reformat

x_reformat=_rnn_reformat(restructured_train_images, 1, 28*28)
x_reformat_test=_rnn_reformat(test_images , 1, 28*28)

In [ ]:
model = MyModel(10)
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x_reformat,train_labels_categorical, batch_size=128, epochs=10, validation_data=(x_reformat_test, test_labels_categorical))
model.summary()
print(model.evaluate(x_reformat_test, test_labels_categorical,verbose=1))

313/313 [==============================] - 16s 48ms/step - loss: 0.1421 - accuracy: 0.9646
[0.1420714557170868, 0.9646000266075134]


#검증실험3

In [ ]:
###유사도 기준 검증실험3
#unit 1기준 랜덤의 유사도 데이터가 unit 2,3,4에 들어감

In [ ]:
# 데이터 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28 * 28)) / 255.0
test_images = test_images.reshape((test_images.shape[0], 28 * 28)) / 255.0

# k-means 클러스터링
kmeans = KMeans(n_clusters=5, random_state=2023)
kmeans.fit(train_images)
cluster_labels = kmeans.labels_

clusters = {}
for i, label in enumerate(cluster_labels):
    if label not in clusters:
        clusters[label] = []
    clusters[label].append(i)

# First, combine all the data from the groups
combined_data = []
for cluster_name in range(5):  # 0, 1, 2, 3, 4
    cluster = clusters[cluster_name]
    for unit in cluster:
        combined_data.append((train_images[unit], train_labels[unit]))

# Now, shuffle the combined data
random.shuffle(combined_data)

# Finally, split the shuffled data back into images and labels
restructured_train_images, restructured_train_labels = zip(*combined_data)

restructured_train_images = np.array(restructured_train_images)
restructured_train_labels = np.array(restructured_train_labels)

# 텐서로 변환
restructured_train_images = tf.convert_to_tensor(restructured_train_images, dtype=tf.float32)
restructured_train_labels = tf.convert_to_tensor(restructured_train_labels, dtype=tf.uint8)
train_labels_categorical = to_categorical(restructured_train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
# 데이터 로드
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((train_images.shape[0], 28 * 28)) / 255.0
test_images = test_images.reshape((test_images.shape[0], 28 * 28)) / 255.0

# 데이터 섞기
combined_data = list(zip(train_images, train_labels))
random.shuffle(combined_data)

restructured_train_images, restructured_train_labels = zip(*combined_data)

restructured_train_images = np.array(restructured_train_images)
restructured_train_labels = np.array(restructured_train_labels)

# 텐서로 변환
restructured_train_images = tf.convert_to_tensor(restructured_train_images, dtype=tf.float32)
restructured_train_labels = tf.convert_to_tensor(restructured_train_labels, dtype=tf.uint8)
train_labels_categorical = to_categorical(restructured_train_labels, num_classes=10)
test_labels_categorical = to_categorical(test_labels, num_classes=10)

In [ ]:
model = MyModel(10)
model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_reformat,train_labels_categorical, batch_size=128, epochs=10, validation_data=(x_reformat_test, test_labels_categorical))

In [ ]:
model.fit(x_reformat,train_labels_categorical, batch_size=128, epochs=10, validation_data=(x_reformat_test, test_labels_categorical))
model.summary()
print(model.evaluate(x_reformat_test, test_labels_categorical))

313/313 [==============================] - 16s 48ms/step - loss: 0.1299 - accuracy: 0.9674
[0.12989245355129242, 0.9674000144004822]
